# Web interface

> Module making it possible to run the conversion from slX-files from a web interface

In [ ]:
#| default_exp webgui

In [ ]:
#| hide
from nbdev.showdoc import *

## Imports and setting up of module

In [ ]:
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from fastlite import *
import fasthtml.components as fh
import httpx
from pathlib import Path

from sonar2csv_shape.convert import *

## Setting styling of web gui

In [ ]:
daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'),
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5/themes.css', rel='stylesheet', type='text/css')
)

In [ ]:
app = FastHTML(hdrs=daisy_hdrs)
rt = app.route

In [ ]:
def get_preview(app):
    return partial(HTMX, app=app, host=None, port=None)
preview = get_preview(app)

::: {.callout-tip collapse="true"}
___
🤔 _Can you please explain to me what the partial function does, how it does that and what its purpose is?_

🤖Reply🤖
The partial function is a really useful tool from Python's functools module. Let me break it down for you:

What it does: partial takes a function and "freezes" some of its arguments, creating a new function with those arguments pre-filled.

How it works: When you call partial(func, arg1, arg2), it returns a new function that, when called, will automatically pass arg1 and arg2 to func, along with any additional arguments you provide later.

Simple example:

```python
def greet(greeting, name):
    return f"{greeting}, {name}!"
```

If you use partial(greet, "Hello"), you get a new function that already has "Hello" as the greeting, so you only need to provide the name later.

In your code, partial(HTMX, app=app, host=None, port=None) creates a new function where app, host, and port are already set. So when you call preview(...) later, those parameters are automatically included.

Does this make sense so far? Would you like me to show you a quick example of how partial works in practice?

:::

In [ ]:
@rt
def testing(): return Div(P('hi'), style='color:red')

In [ ]:
srv = JupyUvi(app=app)

In [ ]:
preview(testing)

## Create upload

See [FastHTML docs - quickstart](https://fastht.ml/docs/tutorials/quickstart_for_web_devs.html#single-file-uploads)

In [ ]:
upload_dir = Path("/app/data")

@rt
def get():
    return Titled("File Upload Demo",
        Article(
            Form(hx_post=upload, hx_target="#result-one")(
                Input(type="file", name="file", cls="file-input file-input-bordered"),
                Button("Upload", type="submit", cls='btn btn-primary'),
            ),
            Div(id="result-one")
        )
    )

def FileMetaDataCard(file):
    return Article(
        Header(H3(file.filename)),
        Ul(
            Li('Size: ', file.size),            
            Li('Content Type: ', file.content_type),
            Li('Headers: ', file.headers),
        )
    )    

@rt
async def upload(file: UploadFile):
    card = FileMetaDataCard(file)
    filebuffer = await file.read()
    (upload_dir / file.filename).write_bytes(filebuffer)
    return card

In [ ]:
preview("get")

## Create download

In [ ]:
@rt
def get_file():
    filepath = "/app/data/solveit_settings.json" # filename
    return FileResponse(filepath)

@rt("/download_page")
def get():
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href="/get_file", cls='btn')
        )
    )

In [ ]:
preview("download_page")

In [ ]:
srv.stop()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()